## Data Preprocessing for Multi Classification Problem

In [ ]:
# Packages/imports here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import spacy
import string
from google.colab import files

%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

pd.options.mode.chained_assignment = None

In [ ]:
#uploaded = files.upload()

In [ ]:
#upload the train dataset
df1 = pd.read_csv('german-contracts-train.csv',  dtype={

        "docid":str,

        "publication_date":str,

        "contract_type":str,

        "nature_of_contract":str,

        "country_code":str,

        "country_name":str,

        "sector":str,

        "category":str,

        "value":float,

        "title":str,

        "description":str,

        "awarding_authority":str,

        "complete_entry":str,

        "label":str   

    })

In [ ]:
#upload the test dataset
df2 = pd.read_csv('german-contracts-test.csv',  dtype={

        "docid":str,

        "publication_date":str,

        "contract_type":str,

        "nature_of_contract":str,

        "country_code":str,

        "country_name":str,

        "sector":str,

        "value":float,

        "title":str,

        "description":str,

        "awarding_authority":str,

        "complete_entry":str,
  

    })

# Data Preprocessing

In [ ]:
df1.shape

(98320, 13)

In [ ]:
df2.shape

(24581, 11)

In [ ]:
#null values in train set provided
df1.isnull().sum()*100/len(df1)

docid                  0.000000
publication_date       0.000000
contract_type          0.000000
nature_of_contract     0.017290
country_code           0.000000
country_name           0.000000
sector                 0.000000
category               0.000000
value                 64.155818
title                  0.000000
description            0.000000
awarding_authority     0.000000
label                  0.000000
dtype: float64

In [ ]:
#null values in test set provided
df2.isnull().sum()*100/len(df2)

docid                  0.000000
publication_date       0.000000
contract_type          0.000000
nature_of_contract     0.004068
country_code           0.000000
country_name           0.000000
sector                 0.000000
value                 64.378992
title                  0.000000
description            0.000000
awarding_authority     0.000000
dtype: float64

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98320 entries, 0 to 98319
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   docid               98320 non-null  object 
 1   publication_date    98320 non-null  object 
 2   contract_type       98320 non-null  object 
 3   nature_of_contract  98303 non-null  object 
 4   country_code        98320 non-null  object 
 5   country_name        98320 non-null  object 
 6   sector              98320 non-null  object 
 7   category            98320 non-null  object 
 8   value               35242 non-null  float64
 9   title               98320 non-null  object 
 10  description         98320 non-null  object 
 11  awarding_authority  98320 non-null  object 
 12  label               98320 non-null  object 
dtypes: float64(1), object(12)
memory usage: 9.8+ MB


In [ ]:
n = 10

l = df1['label'].value_counts()[:n]
l

1000         54520
100000        6753
100           6171
1             5908
100000000     5505
10000         5292
10            2614
1000000       2582
100001000     1459
10000000      1039
Name: label, dtype: int64

In [ ]:
df1_new = df1.copy()

In [ ]:
#keeping the rows without a double 1
df1_new = df1_new.loc[(df1_new['label'] == '1') |
                      (df1_new['label'] == '10') | 
                      (df1_new['label'] == '100') |
                      (df1_new['label'] == '1000') |
                      (df1_new['label'] == '10000') |
                      (df1_new['label'] == '100000') |
                      (df1_new['label'] == '1000000') |
                      (df1_new['label'] == '10000000') |
                      (df1_new['label'] == '100000000')]

In [ ]:
df1_new['label'].value_counts()

1000         54520
100000        6753
100           6171
1             5908
100000000     5505
10000         5292
10            2614
1000000       2582
10000000      1039
Name: label, dtype: int64

In [ ]:
A = len(df1)
A

98320

In [ ]:
B = len(df1_new)
B

90384

In [ ]:
difference = B/A
difference

0.9192839707078926

- we keep the 0.91% of the train dataset by removing 7938 rows

In [ ]:
#concatinating train and test set for preprocessing
df=pd.concat([df1_new, df2],axis=0)

In [ ]:
df.reset_index(inplace = True)

In [ ]:
df.shape

(114965, 14)

In [ ]:
#dropping attribute value, due to amount of null values present in dataset, attribute category not present in test set
df.drop(columns=['value', 'category'], inplace = True)

In [ ]:
df['nature_of_contract'].value_counts()

works       50041
services    41088
supplies    23817
combined        1
Name: nature_of_contract, dtype: int64

In [ ]:
 #replacing null vallues with mode
df['nature_of_contract']=df['nature_of_contract'].fillna(df['nature_of_contract'].mode()[0])

In [ ]:
df.isnull().sum()*100/len(df)

index                  0.00000
docid                  0.00000
publication_date       0.00000
contract_type          0.00000
nature_of_contract     0.00000
country_code           0.00000
country_name           0.00000
sector                 0.00000
title                  0.00000
description            0.00000
awarding_authority     0.00000
label                 21.38129
dtype: float64

## (A) Categorical Feature Handling

In [ ]:
categorical_features=[feature for feature in df.columns if df[feature].dtypes=='O']
categorical_features

['docid',
 'publication_date',
 'contract_type',
 'nature_of_contract',
 'country_code',
 'country_name',
 'sector',
 'title',
 'description',
 'awarding_authority',
 'label']

In [ ]:
for feature in categorical_features:
    print('The feature is {} and number of categories are {}'.format(feature,len(df[feature].unique())))

The feature is docid and number of categories are 114965
The feature is publication_date and number of categories are 504
The feature is contract_type and number of categories are 2
The feature is nature_of_contract and number of categories are 4
The feature is country_code and number of categories are 1
The feature is country_name and number of categories are 1
The feature is sector and number of categories are 1
The feature is title and number of categories are 33671
The feature is description and number of categories are 79137
The feature is awarding_authority and number of categories are 13729
The feature is label and number of categories are 10


In [ ]:
#we decide to drop columns as there exists no variance
df.drop(columns=['country_code','country_name','sector' ], inplace = True)

In [ ]:
#convert strings to numericals for model building 
le = LabelEncoder()

In [ ]:
df['publication_date'] = le.fit_transform(df['publication_date'])
df['contract_type'] = le.fit_transform(df['contract_type'])
df['nature_of_contract'] = le.fit_transform(df['nature_of_contract'])

In [ ]:
df.head()

,index,docid,publication_date,contract_type,nature_of_contract,title,description,awarding_authority,label
0,0,2493527426,114,0,1,Germany-Wilhelmshaven: Cleaning services,Unterhalts- und Glasreinigung.\n,Staatliches Baumanagement Ems-Weser,100000
1,1,2538215982,131,1,1,Germany-Dresden: Engineering-design services f...,ABS Karlsruhe-Stuttgart-Nürnberg-Leipzig/Dresd...,DB Netz AG,1000
2,2,2204943443,100,1,3,"Germany-Germering: Heating, ventilation and ai...",Nach Fertigstellung des ersten Bauabschnitts e...,Große Kreisstadt Germering,1000
3,3,2417769175,96,1,2,Germany-Limbach: Boards,Einrichtung Tafelsystem.\n,Gemeinde Limbach,100000000
4,4,2242098706,93,0,3,Germany-Frankfurt-on-Main: Landscaping work fo...,Im Projekt Neubau Filiale in Dortmund wird das...,"Deutsche Bundesbank, Beschaffungszentrum",1000


In [ ]:
df.shape

(114965, 9)

## (B) Text Features Handling

**Processing attributes:**
- 'title', language = English
- 'description', language = German
- awarding authority', language = German

In [ ]:
#converting the columns from object type into string type and make the words lower casing
def convert_string(column, table):
  return table[column].astype(str).str.lower()


#replacing in the df
df['title'] = convert_string('title', df)
df['description'] = convert_string('description', df)
df['awarding_authority'] = convert_string('awarding_authority', df)

In [ ]:
#remove string of punctuation and digits
PUNCT_TO_REMOVE = string.punctuation + string.digits + '„“' + '—'
print(PUNCT_TO_REMOVE)

def remove_punctuations(text):
    for punctuation in PUNCT_TO_REMOVE:
        text = text.replace(punctuation, ' ')
    return text


#replacing in the df
df["title"] = df['title'].apply(remove_punctuations)
df["description"] = df['description'].apply(remove_punctuations)
df["awarding_authority"] = df['awarding_authority'].apply(remove_punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789„“—


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#stopwords in english
STOPWORDS_ENGLISH = set(stopwords.words('english'))

def remove_stopwords_english(column, table):
    #custom function to remove the stopwords
    return " ".join([word for word in str(column).split() if word not in STOPWORDS_ENGLISH])


#stopwords in german
STOPWORDS_GERMAN = set(stopwords.words('german'))

def remove_stopwords_german(column, table):
    #custom function to remove the stopwords
    return " ".join([word for word in str(column).split() if word not in STOPWORDS_GERMAN])


#replacing in the df
df["title"] = df["title"].apply(lambda title: remove_stopwords_english(title, df))
df["description"] = df["description"].apply(lambda description: remove_stopwords_german(description, df))
df["awarding_authority"] = df["awarding_authority"].apply(lambda awarding_authority: remove_stopwords_german(awarding_authority, df))

In [ ]:
stemmer_english = SnowballStemmer(language='english')
def stem_words_english(text):
    return " ".join([stemmer_english.stem(word) for word in text.split()])

stemmer_german = SnowballStemmer(language='english')
def stem_words_german(text):
    return " ".join([stemmer_german.stem(word) for word in text.split()])

#replacing in the df
df['title'] = df['title'].apply(lambda text: stem_words_english(text))
df['description'] = df['description'].apply(lambda text: stem_words_german(text))
df['awarding_authority'] = df['awarding_authority'].apply(lambda text: stem_words_german(text))

In [ ]:
nltk.download('punkt')

stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

In [ ]:
clean_text("Python ist die beste Programmiersprache der Welt.")

'python best programmiersprach welt'

In [ ]:
#replacing in the df
df['description'] = df['description'].apply(lambda text: clean_text(text))
df['awarding_authority'] = df['awarding_authority'].apply(lambda text: clean_text(text))

In [ ]:
df.head()

,index,docid,publication_date,contract_type,nature_of_contract,title,description,awarding_authority,label
0,0,2493527426,114,0,1,germani wilhelmshaven clean servic,unterhalt glasrein,staatlich baumanag em wes,100000
1,1,2538215982,131,1,1,germani dresden engin design servic traffic in...,ab karlsruh stuttgart nurnberg leipzig dresd b...,db netz ag,1000
2,2,2204943443,100,1,3,germani germer heat ventil air condit instal work,fertigstell erst bauabschnitt erfolgt zweit ba...,gross kreisstadt germ,1000
3,3,2417769175,96,1,2,germani limbach board,einricht tafelsyst,gemeind limbach,100000000
4,4,2242098706,93,0,3,germani frankfurt main landscap work green area,projekt neubau filial dortmund gewerk galabau ...,deutsch bundesbank beschaffungszentrum,1000


In [ ]:
df.shape

(114965, 9)

In [ ]:
#Extract dataset for the multi classification task without data augmentation
df.to_csv('dataset_task2.csv')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')